In [39]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np

### ● 不動産の一つの物件から情報を取得する。

In [40]:
base_url = "https://www.chintai.net/list/?rt=50&prefKey=tokyo&mode=1&action=ohaza&prefKey=tokyo&g=13101&g=13102&g=13103&g=13104&g=13105&g=13109&g=13110&g=13111&g=13113&urlType=dynamic&cf=0&ct=0&k=1&jk=0&jl=0&sf=0&st=0&j=&h=99&b=1&b=2&b=3&jks="
# とりあえず都心＆南部（東京都千代田区、東京都中央区、東京都港区、東京都新宿区、東京都文京区、東京都渋谷区、東京都品川区、東京都目黒区、東京都大田区、東京都世田谷区）で設定

response = requests.get(base_url)
soup     = BeautifulSoup(response.content, "lxml")
items    = soup.find(class_="cassette_item build")

In [41]:
# スクレイピングした HTML情報出力
f = open("test.txt", "w")
f.write(str(items))
f.close()

In [49]:
# 各物件情報の取得
property_name = items.find(class_="cassette_ttl ttl_main").find('h2').contents[1].strip() if items.find(class_="cassette_ttl ttl_main") else None
category    = items.find('span', class_='icn_typeB').get_text() if items.find('span', class_='icn_typeB') else None
# address_th = items.find('th', string='住所')
address = address_th.find_next_sibling('td').get_text().replace('周辺地図', '').strip() if address_th and address_th.find_next_sibling('td') else None
nearest_stations = [station.get_text(strip=True).replace('\xa0', ' ') for station in items.find('th', class_='traffic').find_next_sibling('td').find_all('li')] if items.find('th', class_='traffic').find_next_sibling('td') else []
years_since_const = re.search(r'(築\d+年)', items.find('th', string='築年').find_next_sibling('td').get_text(strip=True)).group(1) if items.find('th', string='築年') else None
number_of_floors = items.find('th', string='階建').find_next_sibling('td').get_text(strip=True) if items.find('th', string='階建') else None
floor_number = items.find('td', class_='floar').find('li').get_text(strip=True) if items.find('td', class_='floar') else None
rent_admin_fee = f"{items.find('td', class_='price').find('span', class_='num').get_text(strip=True) + '万円'} / {items.find('td', class_='price').get_text(strip=True).split('円')[1]}円" if items.find('td', class_='price') and items.find('td', class_='price').find('span', class_='num') else None
deposit_gratuity = items.find('td', class_='other_price').get_text(separator=' / ', strip=True) if items.find('td', class_='other_price') else None
layout_total_area = next(
    (
        f"{layout} / {area}"
        for td in items.find_all('td', class_='clickArea')
        if "m²" in (text := td.get_text(strip=True))
        if (layout := re.search(r'^\d*[A-Z]+', text)) and (area := re.search(r'\d+\.\d+m²', text))
        if (layout := layout.group(0)) and (area := area.group(0))
    ),
    None
)



# 各物件情報の表示
print("物件名称 (Property Name):", property_name)
print("カテゴリー (Category):", category)
print("住所 (Address):", address)
print("最寄り駅 (Nearest Stations):", nearest_stations)
print("築年数 (Years Since Construction):", years_since_const)
print("階建 (Number of Floors):", number_of_floors)
print("階数 (Floor Number):", floor_number)
print("賃料/管理費 (Rent/Administration Fee):", rent_admin_fee)
print("敷金/礼金 (Deposit/Key Money):", deposit_gratuity)
print("間取り/占有面積 (Layout/Total Area):", layout_total_area)

物件名称 (Property Name): クリスタル自由が丘
カテゴリー (Category): 賃貸ハイツ
住所 (Address): 東京都目黒区緑が丘２丁目
最寄り駅 (Nearest Stations): ['東急東横線/自由が丘駅 徒歩5分', '東急大井町線/緑が丘駅 徒歩11分', '東急目黒線/奥沢駅 徒歩10分']
築年数 (Years Since Construction): 築38年
階建 (Number of Floors): 2階建
階数 (Floor Number): 2階
賃料/管理費 (Rent/Administration Fee): 6.3万円 / 2,000円
敷金/礼金 (Deposit/Key Money): 6.3万 / --
間取り/占有面積 (Layout/Total Area): 1R / 15.52m²


In [64]:
# 物件画像・間取り画像・詳細URLの取得
property_image_element = items.find(class_="photo_box picBox")
property_image_url = property_image_element.img["data-original"] if property_image_element and property_image_element.img else None

# URLを絶対パスに変換
if property_image_url and property_image_url.startswith("//"):
    property_image_url = "https:" + property_image_url

floor_plan_image_element = items.find(class_="plan-box")
floor_plan_image_url = floor_plan_image_element.img["data-original"] if floor_plan_image_element and floor_plan_image_element.img else None

# URLを絶対パスに変換
if floor_plan_image_url and floor_plan_image_url.startswith("//"):
    floor_plan_image_url = "https:" + floor_plan_image_url

# 物件リンクの抽出
property_link_element = items.find('a', {'data-detailurl': True})
property_link = property_link_element['data-detailurl'] if property_link_element else None

# URLを絶対パスに変換
if property_link and property_link.startswith("/"):
    property_link = "https://www.chintai.net" + property_link

# # 物件画像・間取り画像・詳細URLの表示
print("物件画像 URL (Property Image URL):", property_image_url)
print("間取り情報画像 URL (Floor Plan Image URL):", floor_plan_image_url)
print("物件リンク (Property Link):", property_link)

物件画像 URL (Property Image URL): https://img.chintai.net/awsorg/bk/1000/60/38/000000000716038/021/0000000007160380006_20240525114833732.JPG?interpolation=lanczos-none&resize=210:*
間取り情報画像 URL (Floor Plan Image URL): https://img.chintai.net/awsorg/bk/1000/60/38/000000000716038/011/0000000007160380006_20240525114833351.JPG?interpolation=lanczos-none&resize=115:*
物件リンク (Property Link): https://www.chintai.net/detail/bk-0000000360000000007160380006/?sidesFlg=1


### ● 不動産の複数ページから情報を取得する。

In [ ]:
# 基本URLと最大ページ数の設定
base_url = "https://www.chintai.net/list/?rt=50&prefKey=tokyo&mode=1&action=ohaza&prefKey=tokyo&g=13101&g=13102&g=13103&g=13104&g=13105&g=13109&g=13110&g=13111&g=13113&urlType=dynamic&cf=0&ct=0&k=1&jk=0&jl=0&sf=0&st=0&j=&h=99&b=1&b=2&b=3&jks="
# とりあえず都心＆南部（東京都千代田区、東京都中央区、東京都港区、東京都新宿区、東京都文京区、東京都渋谷区、東京都品川区、東京都目黒区、東京都大田区、東京都世田谷区）で設定

max_page = 5  # 最大ページ数

all_data = []

for page in range(1, max_page + 1):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')
    items = soup.findAll("div", {"class": "cassette_item build"})


    print("page", page, "items", len(items))

    for item in items:
        base_data = {}
        base_data["名称"]     = items.find(class_="cassette_ttl ttl_main").find('h2').contents[1].strip() if items.find(class_="cassette_ttl ttl_main") else None
        base_data["カテゴリ"] = items.find('span', class_='icn_typeB').get_text() if items.find('span', class_='icn_typeB') else None
        base_data["アドレス"] = 
        
        # 駅のアクセス情報をまとめて取得
        base_data["アクセス"] = ", ".join([station.get_text(strip=True) for station in item.findAll("div", {"class": "cassetteitem_detail-text"})])

        construction_info = item.find("li", {"class": "cassetteitem_detail-col3"}).find_all("div") if item.find("li", {"class": "cassetteitem_detail-col3"}) else None
        base_data["築年数"] = construction_info[0].get_text(strip=True) if construction_info and len(construction_info) > 0 else None
        base_data["構造"] = construction_info[1].get_text(strip=True) if construction_info and len(construction_info) > 1 else None

        tbodys = item.find("table", {"class": "cassetteitem_other"}).findAll("tbody")

        for tbody in tbodys:
            data = base_data.copy()
            # 階数情報の正確な取得
            floor_info = tbody.find_all("td")[2].get_text(strip=True) if len(tbody.find_all("td")) > 2 else None
            data["階数"]   = floor_info
            data["家賃"]   = tbody.select_one(".cassetteitem_price--rent").get_text(strip=True) if tbody.select_one(".cassetteitem_price--rent") else None
            data["管理費"] = tbody.select_one(".cassetteitem_price--administration").get_text(strip=True) if tbody.select_one(".cassetteitem_price--administration") else None
            data["敷金"]   = tbody.select_one(".cassetteitem_price--deposit").get_text(strip=True) if tbody.select_one(".cassetteitem_price--deposit") else None
            data["礼金"]   = tbody.select_one(".cassetteitem_price--gratuity").get_text(strip=True) if tbody.select_one(".cassetteitem_price--gratuity") else None
            data["間取り"] = tbody.select_one(".cassetteitem_madori").get_text(strip=True) if tbody.select_one(".cassetteitem_madori") else None
            data["面積"]   = tbody.select_one(".cassetteitem_menseki").get_text(strip=True) if tbody.select_one(".cassetteitem_menseki") else None

            # 物件画像・間取り画像・詳細URLの取得を最後に行う
            property_image_element = item.find(class_="cassetteitem_object-item")
            data["物件画像URL"] = property_image_element.img["rel"] if property_image_element and property_image_element.img else None

            floor_plan_image_element = item.find(class_="casssetteitem_other-thumbnail")
            data["間取画像URL"] = floor_plan_image_element.img["rel"] if floor_plan_image_element and floor_plan_image_element.img else None

            property_link_element = item.select_one("a[href*='/chintai/jnc_']")
            data["物件詳細URL"] = "https://suumo.jp" +property_link_element['href'] if property_link_element else None ## 不動産サイトから詳細URLリンクを読み解き作成

            all_data.append(data)    


In [ ]:
df = pd.DataFrame(all_data)
df = df.drop_duplicates() # 重複データの削除
df.head(2)

In [ ]:
df.tail(2)

### ● Googleスプレッドシートへの書き込み・読み込み。

In [ ]:
# google スプレッドシート 書き込み・読み込み
import gspread
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe
from gspread_dataframe import set_with_dataframe

In [ ]:
from dotenv import load_dotenv
import os

# 環境変数の読み込み
load_dotenv()

# 環境変数から認証情報を取得
SPREADSHEET_ID = "1_JALV0dH6ROPgKKMgjD_yizCqGzY_woSLxXxAVjaftw"
PRIVATE_KEY_PATH = "/Users/simon/Realestate_Search/齋藤作業用/realestatesearch-yokopisans160-47c75af30d9f.json"

# 一旦齋藤の情報をベタで入れています。

In [ ]:
PRIVATE_KEY_PATH

In [ ]:
# googleスプレッドシートの認証 jsonファイル読み込み(key値はGCPから取得)
SP_CREDENTIAL_FILE = PRIVATE_KEY_PATH

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    SP_CREDENTIAL_FILE,
    scopes=scopes
)
gc = gspread.authorize(credentials)


SP_SHEET_KEY = SPREADSHEET_ID # d/〇〇/edit の〇〇部分
sh  = gc.open_by_key(SP_SHEET_KEY)

In [ ]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, df)

In [ ]:
# 不動産データの取得
SP_SHEET     = 'tech0' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [ ]:
new_df.info()

In [ ]:
new_df.head(2)

In [ ]:
new_df['築年数'] = new_df["築年数"].apply( lambda x: 0 if x=='新築' else int(re.split('[築年]', x )[1]) )

In [ ]:
def get_most_floor(x):
    if ('階建' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階建',str(x))
        list = map(int, list)
        min_value = min(list)
        return min_value

new_df['構造'] = new_df['構造'].apply(get_most_floor)
print(new_df['構造'].head(5))

In [ ]:
def get_floor(x):
    if ('階' not in x) :
        return np.nan
    elif('B' not in x) :
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = min(list)
        return min_value
    else:
        list = re.findall(r'(\d+)階',str(x))
        # time_listを数値型に変換
        list = map(int, list)
        # time_listの最小値をmin_valueに代入
        min_value = -1*min(list)
        return min_value

new_df['階数'] = new_df['階数'].apply(get_floor)
print(new_df['階数'].head(5))

In [ ]:
def change_fee(x):
    if ('万円' not in x) :
        return np.nan
    else:
        return float(x.split('万円')[0])

new_df['家賃'] = new_df['家賃'].apply(change_fee)
new_df['敷金'] = new_df['敷金'].apply(change_fee)
new_df['礼金'] = new_df['礼金'].apply(change_fee)

In [ ]:
new_df.head(5)

In [ ]:
def change_fee2(x):
    if ('円' not in x) :
        return np.nan
    else:
        return float(x.split('円')[0])


new_df['管理費'] = new_df['管理費'].apply(change_fee2)

In [ ]:
new_df['面積'] = new_df['面積'].apply(lambda x: float(x[:-2]))

In [ ]:
new_df['区'] = new_df["アドレス"].apply(lambda x : x[x.find("都")+1:x.find("区")+1])

In [ ]:
new_df['市町'] = new_df["アドレス"].apply(lambda x : x[x.find("区")+1 :-1])

In [ ]:
def split_access(row):
    accesses = row['アクセス'].split(', ')
    results = {}

    for i, access in enumerate(accesses, start=1):
        if i > 3:
            break  # 最大3つのアクセス情報のみを考慮

        parts = access.split('/')
        if len(parts) == 2:
            line_station, walk = parts
            # ' 歩'で分割できるか確認
            if ' 歩' in walk:
                station, walk_min = walk.split(' 歩')
                # 歩数の分の数値だけを抽出
                walk_min = int(re.search(r'\d+', walk_min).group())
            else:
                station = None
                walk_min = None
        else:
            line_station = access
            station = walk_min = None

        results[f'アクセス①{i}線路名'] = line_station
        results[f'アクセス①{i}駅名'] = station
        results[f'アクセス①{i}徒歩(分)'] = walk_min

    return pd.Series(results)

# 新しい列をデータフレームに適用
new_df = new_df.join(new_df.apply(split_access, axis=1))

In [ ]:
new_df.head(2)

In [ ]:
new_df.info()

In [ ]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_2' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)

### ● 緯度経度情報取得

In [ ]:
# 不動産データの取得
SP_SHEET     = 'tech0_2' # sheet名
worksheet = sh.worksheet(SP_SHEET) # シートのデータ取得
pre_data  = worksheet.get_all_values()
col_name = pre_data[0][:]
new_df = pd.DataFrame(pre_data[1:], columns=col_name) # 一段目をカラム、以下データフレームで取得

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import pandas as pd
import time

# ジオコーダーの初期化
geolocator = Nominatim(user_agent="your_app_name", timeout=10)

current_count = 0
total_count = len(new_df['アドレス'])

# 住所から緯度と経度を取得する関数
def get_lat_lon(address, retries=3):
    global current_count
    current_count += 1
    
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                print(f"{current_count}/{total_count} 件目実施中 結果: {location.latitude}, {location.longitude}")
                return location.latitude, location.longitude
            else:
                print(f"{current_count}/{total_count} 件目実施中 結果: 住所が見つかりません")
                return None, None
        except (GeocoderTimedOut, GeocoderServiceError) as e:
            print(f"Error retrieving location for address {address}: {e}. Retrying ({attempt + 1}/{retries})...")
            time.sleep(1)  # リトライの前に1秒待つ

    print(f"Failed to retrieve location for address {address} after {retries} retries")
    return None, None

# 新しい列を作成してデータフレームに追加
new_df['緯度'], new_df['経度'] = zip(*new_df['アドレス'].apply(get_lat_lon))

# 結果の表示
print(new_df.head())

# 単一の住所のテスト
address = "東京都江戸川区篠崎町７"
location = geolocator.geocode(address)
if location:
    print(f"テストアドレスの結果: {location.latitude}, {location.longitude}")
else:
    print("テストアドレスの結果: 住所が見つかりません")

In [ ]:
# 取得した不動産データの書き込み
SP_SHEET_wr     = 'tech0_3' # sheet名
worksheet_wr = sh.worksheet(SP_SHEET_wr) # シートのデータ取得
set_with_dataframe(worksheet_wr, new_df)